In [12]:
# !pip install pandas
# !pip install plotly
# !pip install ipywidgets
# !pip install google.colab

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider

In [14]:
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
file_path = 'weather - 286_40.75_t2m_1d.csv'
weather_df = pd.read_csv(file_path)
weather_df.head()

,time,longitude,latitude,Ktemp
0,1950-01-01 9:00:00,286,40.75,274.39734
1,1950-01-02 9:00:00,286,40.75,277.07593
2,1950-01-03 9:00:00,286,40.75,280.80090
3,1950-01-04 9:00:00,286,40.75,287.14010
4,1950-01-05 9:00:00,286,40.75,285.65262


In [16]:
weather_df['time'] = pd.to_datetime(weather_df['time'])
weather_df['year'] = weather_df['time'].dt.year
weather_df['month'] = weather_df['time'].dt.month

In [17]:
weather_df['Ftemp'] = (weather_df['Ktemp'] - 273.15) * (9/5) + 32

monthly_avg_temp = weather_df.groupby(['year', 'month'])['Ftemp'].mean().reset_index()

In [18]:
def plot_avg_monthly_temps(year):
    """Plot the average monthly temperatures for a given year."""
    plt.figure(figsize=(10, 6))
    yearly_data = monthly_avg_temp[monthly_avg_temp['year'] == year]
    plt.plot(yearly_data['month'], yearly_data['Ftemp'], marker='o', linestyle='-')
    plt.title(f'Average Monthly Temperatures in {year}')
    plt.xlabel('Month')
    plt.ylabel('Temperature (°F)')
    plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    plt.grid(True)
    plt.show()

In [19]:
year_slider = IntSlider(min=weather_df['year'].min(), max=weather_df['year'].max(), step=1, value=weather_df['year'].min(), description='Select Year')

interact(plot_avg_monthly_temps, year=year_slider)

interactive(children=(IntSlider(value=1950, description='Select Year', max=2021, min=1950), Output()), _dom_cl…

<function __main__.plot_avg_monthly_temps(year)>

In [20]:
annual_avg_temp = weather_df.groupby('year')['Ftemp'].mean().reset_index()
warm_year = annual_avg_temp[annual_avg_temp['Ftemp'] > 55].head(1)

if not warm_year.empty:
    first_warm_year = warm_year['year'].values[0]
    print(f"The first year where the average temperature exceeded 55 degrees Fahrenheit was {first_warm_year}.")
else:
    print("No year in the dataset has an average temperature exceeding 55 degrees Fahrenheit.")

The first year where the average temperature exceeded 55 degrees Fahrenheit was 1953.


In [21]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

In [22]:
def prepare_seasonal_data(df):
    df['season'] = df['month'].apply(get_season)
    seasonal_avg_temp = df.groupby(['year', 'season'])['Ftemp'].mean().reset_index()
    return seasonal_avg_temp.pivot(index='year', columns='season', values='Ftemp')

def plot_data_with_matplotlib(start_year, seasonal_avg_temp_pivot):
    seasons = ['Winter', 'Spring', 'Summer', 'Fall']
    colors = ['blue', 'green', 'red', 'orange']

    plt.figure(figsize=(10, 6))
    for season, color in zip(seasons, colors):
        plt.plot(seasonal_avg_temp_pivot.loc[start_year:start_year+9].index,
                 seasonal_avg_temp_pivot.loc[start_year:start_year+9, season],
                 marker='o', color=color, label=season)

    plt.title(f'Seasonal Average Temperatures from {start_year} to {start_year+9}')
    plt.xlabel('Year')
    plt.ylabel('Temperature (F)')
    plt.legend(title='Season')
    plt.grid(True)
    plt.show()

def interactive_plot(seasonal_avg_temp_pivot):
    start_year = seasonal_avg_temp_pivot.index.min()
    end_year = seasonal_avg_temp_pivot.index.max()

    @interact
    def update_plot(start_year=IntSlider(min=start_year, max=end_year-10, step=1, value=start_year, description='Decade Start:')):
        plot_data_with_matplotlib(start_year, seasonal_avg_temp_pivot)

seasonal_avg_temp_pivot = prepare_seasonal_data(weather_df)

interactive_plot(seasonal_avg_temp_pivot)

interactive(children=(IntSlider(value=1950, description='Decade Start:', max=2011, min=1950), Output()), _dom_…